In [77]:
!pip install xmltodict
!pip install beautifulsoup4

## 공공데이터포털에서 open API불러오기

python 버전 3에서는 urllib2가 없으므로 "from urillib2 import Request, urlopen" 코드는 오류 발생
"from urllib.request import Request, urlopen" 이용

또한 "from urllib import urlencode, quote_plus"도 오류 발생
"from urllib.parse import urlencode, quote_plus" 입력

In [209]:
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus

url = 'http://openapi.q-net.or.kr/api/service/rest/InquiryStatSVC/getGradPiPassList'
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : 'ygMvNxJQ6HFnIUQtG0H5ykqHK9ANGqXm9AfnHa75GILHvyP4bsRRZfQqfpXtrn+3WmN+kyGCLOLqGo0qlF8HQg==', quote_plus('baseYY') : '2017' })

request = Request(url + queryParams)
request.get_method = lambda: 'GET'
response_body= urlopen(request).read()

In [211]:
type(response_body)

bytes

In [212]:
response_body

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><gradename>\xea\xb8\xb0\xec\x88\xa0\xec\x82\xac</gradename><statisyy1>59</statisyy1><statisyy2>56</statisyy2><statisyy3>55</statisyy3><statisyy4>53</statisyy4><statisyy5>56</statisyy5><statisyy6>65</statisyy6></item><item><gradename>\xea\xb8\xb0\xeb\x8a\xa5\xec\x9e\xa5</gradename><statisyy1>33</statisyy1><statisyy2>36</statisyy2><statisyy3>24</statisyy3><statisyy4>23</statisyy4><statisyy5>30</statisyy5><statisyy6>29</statisyy6></item><item><gradename>\xea\xb8\xb0\xec\x82\xac</gradename><statisyy1>41</statisyy1><statisyy2>43</statisyy2><statisyy3>42</statisyy3><statisyy4>37</statisyy4><statisyy5>40</statisyy5><statisyy6>38</statisyy6></item><item><gradename>\xec\x82\xb0\xec\x97\x85\xea\xb8\xb0\xec\x82\xac</gradename><statisyy1>47</statisyy1><statisyy2>49</statisyy2><statisyy3>46</statisyy3><statisyy4>49</statisyy4><statis

### Decoding
utf-8 형식은 byte 타입으로 인코딩 되어 있는 상태.
이를 문자열로 변환하는 것을 decode라 함.

https://chrisjune-13837.medium.com/%EC%9D%B8%EC%BD%94%EB%94%A9%EA%B3%BC-%EB%94%94%EC%BD%94%EB%94%A9-87006cf8dee2

In [213]:
decode_response= response_body.decode('utf-8')

In [185]:
type(response_body)

bytes

In [186]:
type(decode_response)

str

### method 1) BeautifulSoup
BeautifulSoup의 라이브러리 내에 있는 html parser는 xml의 구조를 파악함
html parser를 이용해 xml파일 형태를 인식함으로써 item을 찾을 수 있음(?)

++ 추가 자료 조사 필요
https://godongyoung.github.io/%EC%BD%94%EB%94%A9cheat%20sheet/2019/06/02/Open-API-%ED%99%9C%EC%9A%A9%EB%B0%A9%EB%B2%95-%EC%A0%95%EB%A6%AC(with-Python).html

In [214]:
from bs4 import BeautifulSoup

soup= BeautifulSoup(decode_response, 'html.parser')

In [215]:
datas_bs= soup.find_all('item')
datas_bs

[<item><gradename>기술사</gradename><statisyy1>59</statisyy1><statisyy2>56</statisyy2><statisyy3>55</statisyy3><statisyy4>53</statisyy4><statisyy5>56</statisyy5><statisyy6>65</statisyy6></item>,
 <item><gradename>기능장</gradename><statisyy1>33</statisyy1><statisyy2>36</statisyy2><statisyy3>24</statisyy3><statisyy4>23</statisyy4><statisyy5>30</statisyy5><statisyy6>29</statisyy6></item>,
 <item><gradename>기사</gradename><statisyy1>41</statisyy1><statisyy2>43</statisyy2><statisyy3>42</statisyy3><statisyy4>37</statisyy4><statisyy5>40</statisyy5><statisyy6>38</statisyy6></item>,
 <item><gradename>산업기사</gradename><statisyy1>47</statisyy1><statisyy2>49</statisyy2><statisyy3>46</statisyy3><statisyy4>49</statisyy4><statisyy5>45</statisyy5><statisyy6>44</statisyy6></item>,
 <item><gradename>전문사무</gradename><statisyy1>40</statisyy1><statisyy2>41</statisyy2><statisyy3>36</statisyy3><statisyy4>34</statisyy4><statisyy5>34</statisyy5><statisyy6>30</statisyy6></item>,
 <item><gradename>기능사</gradename><stati

In [216]:
type(datas_bs)

bs4.element.ResultSet

In [217]:
# .text를 통해 데이터 구조말고 내용만 보기

[x.text for x in datas_bs][:5]

['기술사595655535665',
 '기능장333624233029',
 '기사414342374038',
 '산업기사474946494544',
 '전문사무404136343430']

### method 2) bytes형태에서 str형태로 바꾼 xml을  dict형태로 변환

In [2]:
import xmltodict

to_json= xmltodict.parse(decode_response)
to_dict= json.dumps(to_json)

datas_dict= json.loads(to_dict)

NameError: name 'decode_response' is not defined

In [248]:
tmp=to_json['response']['body']['items']['item']
tmp[0]

OrderedDict([('gradename', '기술사'),
             ('statisyy1', '59'),
             ('statisyy2', '56'),
             ('statisyy3', '55'),
             ('statisyy4', '53'),
             ('statisyy5', '56'),
             ('statisyy6', '65')])

In [233]:
datas_dict=datas_dict['response']['body']['items']['item']

datas_dict

[{'gradename': '기술사',
  'statisyy1': '59',
  'statisyy2': '56',
  'statisyy3': '55',
  'statisyy4': '53',
  'statisyy5': '56',
  'statisyy6': '65'},
 {'gradename': '기능장',
  'statisyy1': '33',
  'statisyy2': '36',
  'statisyy3': '24',
  'statisyy4': '23',
  'statisyy5': '30',
  'statisyy6': '29'},
 {'gradename': '기사',
  'statisyy1': '41',
  'statisyy2': '43',
  'statisyy3': '42',
  'statisyy4': '37',
  'statisyy5': '40',
  'statisyy6': '38'},
 {'gradename': '산업기사',
  'statisyy1': '47',
  'statisyy2': '49',
  'statisyy3': '46',
  'statisyy4': '49',
  'statisyy5': '45',
  'statisyy6': '44'},
 {'gradename': '전문사무',
  'statisyy1': '40',
  'statisyy2': '41',
  'statisyy3': '36',
  'statisyy4': '34',
  'statisyy5': '34',
  'statisyy6': '30'},
 {'gradename': '기능사',
  'statisyy1': '49',
  'statisyy2': '48',
  'statisyy3': '48',
  'statisyy4': '47',
  'statisyy5': '47',
  'statisyy6': '51'},
 {'gradename': '기능사보', 'statisyy6': '64'}]

In [238]:
df_cols =[]

for key, val in datas_dict[0].items():
    df_cols.append(key)
    
print(df_cols)

['gradename', 'statisyy1', 'statisyy2', 'statisyy3', 'statisyy4', 'statisyy5', 'statisyy6']


In [250]:
type(datas_dict)

list

In [242]:
df_rows={}
out_df= pd.DataFrame(columns=df_cols)

for i in range(len(datas_dict)):
    for key, val in datas_dict[i].items():
        df_rows[key]= val
    
    print(df_rows)
    out_df= out_df.append(df_rows, ignore_index=True)

{'gradename': '기술사', 'statisyy1': '59', 'statisyy2': '56', 'statisyy3': '55', 'statisyy4': '53', 'statisyy5': '56', 'statisyy6': '65'}
{'gradename': '기능장', 'statisyy1': '33', 'statisyy2': '36', 'statisyy3': '24', 'statisyy4': '23', 'statisyy5': '30', 'statisyy6': '29'}
{'gradename': '기사', 'statisyy1': '41', 'statisyy2': '43', 'statisyy3': '42', 'statisyy4': '37', 'statisyy5': '40', 'statisyy6': '38'}
{'gradename': '산업기사', 'statisyy1': '47', 'statisyy2': '49', 'statisyy3': '46', 'statisyy4': '49', 'statisyy5': '45', 'statisyy6': '44'}
{'gradename': '전문사무', 'statisyy1': '40', 'statisyy2': '41', 'statisyy3': '36', 'statisyy4': '34', 'statisyy5': '34', 'statisyy6': '30'}
{'gradename': '기능사', 'statisyy1': '49', 'statisyy2': '48', 'statisyy3': '48', 'statisyy4': '47', 'statisyy5': '47', 'statisyy6': '51'}
{'gradename': '기능사보', 'statisyy1': '49', 'statisyy2': '48', 'statisyy3': '48', 'statisyy4': '47', 'statisyy5': '47', 'statisyy6': '64'}


In [243]:
out_df

,gradename,statisyy1,statisyy2,statisyy3,statisyy4,statisyy5,statisyy6
0,기술사,59,56,55,53,56,65
1,기능장,33,36,24,23,30,29
2,기사,41,43,42,37,40,38
3,산업기사,47,49,46,49,45,44
4,전문사무,40,41,36,34,34,30
5,기능사,49,48,48,47,47,51
6,기능사보,49,48,48,47,47,64
